In [ ]:
import numpy as np
import pandas as pd

### Monte Carlo

In [ ]:
def Monte_Carlo():
    mean_arr = -np.ones(d['rep_num'])
    max_list = []

    for i in range(d['rep_num']):
        chosen_matrix = np.zeros((d['sim_num'], d['n']+2))
        chosen_matrix[:,0] = np.log(d['Smax_t'])
        chosen_matrix[:,1] = np.log(d['St'])

        S_change_mean = (d['r']-d['q']-(d['sigma']**2)/2)*d['delta_t']
        S_std = d['sigma'] * np.sqrt(d['delta_t'])

        for t in range(2, d['n']+2):
          change_arr = np.random.normal(S_change_mean, S_std, d['sim_num'])
          chosen_matrix[:, t] = chosen_matrix[:, t-1] + change_arr
        chosen_matrix = np.exp(chosen_matrix)
        max_list = np.max(chosen_matrix, axis=1)
        payoff = max_list - chosen_matrix[:,d['n']+1]
        mean_arr[i] = payoff.mean() *np.exp(-d['r'] * (d['T'] - d['t']))
    if any(m < 0 for m in mean_arr):
        print("There is negative element in mean_arr.")
    return mean_arr.mean(), mean_arr.std()

### Tree

In [ ]:
class S_node():
    def __init__(self, S):
        self.S = S
        self.max_list = []
        self.value_EP = []
        self.value_AP = []
    def set_max_list(self, max_list):
        self.max_list = max_list
    def update_max_list(self,parent_max_list): #parent_max_list is decreasing
        temp = np.array(parent_max_list)
        to_add = temp[temp >= self.S].tolist()
        self.max_list += to_add
        if (self.S not in self.max_list) and (len(to_add)<len(temp)): # < represents that there are elements being removed, so add S into the max_list
            self.max_list.append(self.S)
        self.max_list = list(set(self.max_list))
        self.max_list.sort(reverse = True)
    def calculate_payoff(self, uparent_node, dparent_node, p, discount_factor, terminal = False): ##
        if terminal == True:
          for i in range(len(self.max_list)):
            self.value_EP.append(np.max(self.max_list[i]- self.S, 0))
            self.value_AP.append(np.max(self.max_list[i]- self.S, 0))
        else:
          for k in range(len(self.max_list)):
            # print("k = ",k)
            added = False
            for i in range(len(uparent_node.max_list)):
              # print("self maxlist = ", self.max_list[0], ", up parent max first = ",uparent_node.max_list[0])
              # while self.max_list[k] <= uparent_node.max_list[i]:
              # print("i = ",i)
              if abs(self.max_list[k] - uparent_node.max_list[i]) < 1e-12:
                up_put_EP = uparent_node.value_EP[i]
                up_put_AP = uparent_node.value_AP[i]
                added = True
                break
            if not added: # not in max_list
              index = uparent_node.max_list.index(uparent_node.S)
              up_put_EP = uparent_node.value_EP[index]
              up_put_AP = uparent_node.value_AP[index]
              added = True
            if not added:
              print("Not finding corresponding value from up parent node.")

            #print("up put EP = %4f, up put AP = %4f" % (up_put_EP, up_put_AP))


            added = False
            # print("Start point: S = %f and dparent first max = %f" % (self.max_list[k], dparent_node.max_list[i]))
            for i in range(len(dparent_node.max_list)):
              # while self.max_list[k] <= dparent_node.max_list[i]:
              # print("S max = ", self.max_list[k], ", max_list[i] = ", dparent_node.max_list[i])
              if abs(self.max_list[k] - dparent_node.max_list[i]) < 1e-12:
                d_put_EP = dparent_node.value_EP[i]
                d_put_AP = dparent_node.value_AP[i]
                added = True
                break
            if not added:
              print("Not finding corresponding value from down parent node.")

            #print("down put EP = %4f, down put AP = %4f" % (d_put_EP, d_put_AP))
            value_EP = (p * up_put_EP + (1 - p) * d_put_EP )* discount_factor
            self.value_EP.append(value_EP)
            value_AP = np.max([self.max_list[k] - self.S, (p * up_put_AP + (1 - p) * d_put_AP )* discount_factor])
            self.value_AP.append(value_AP)
            #print("value_EP = %f, value_AP = %f" % (value_EP, value_AP))

    def print(self):
        print("S = %4f" % self.S)
        print("S_max list = ", end = '')
        print(self.max_list)
        print(self.value_EP)
        print(self.value_AP)

In [ ]:
class Binomial_tree():
    def __init__(self):
        self.u = np.exp(d['sigma'] * np.sqrt(d['delta_t']))
        self.d = np.exp(-d['sigma'] * np.sqrt(d['delta_t']))
        self.p = (np.exp((d['r'] -d['q'])*d['delta_t']) - self.d)/(self.u - self.d)

    def build_tree(self):
        self.nodes = []
        start_node = S_node(d['St'])
        start_node.set_max_list([np.max([d['St'], d['Smax_t']])])
        self.nodes.append([start_node])

        u_list = []
        d_list = []
        for i in range(d['n']):
            u_list.append(self.u ** (i+1))
            d_list.append(self.d ** (i+1))

        for i in range(1, d['n']+1):
            self.nodes.append([])
            for j in range(i+1):
                a = i-j # new_S = St * u**(i-j) * d**(j)}
                b = j
                if a > b:
                    new_S = u_list[a-b-1] * d['St']
                elif a < b:
                    new_S = d_list[b-a-1] * d['St']
                else: # a==b
                    new_S = d['St']

                new_node = S_node(new_S)
                if j==0:
                    new_node.update_max_list(self.nodes[i-1][0].max_list)
                elif j==i:
                    new_node.update_max_list(self.nodes[i-1][i-1].max_list)
                else:
                    new_node.update_max_list(self.nodes[i-1][j-1].max_list)
                    new_node.update_max_list(self.nodes[i-1][j].max_list)

                self.nodes[i].append(new_node)
                # (self.nodes[i][j]).print()
    def build_tree_1(self):
        self.nodes = []
        start_node = S_node(d['St'])
        start_node.set_max_list([np.max([d['St'], d['Smax_t']])])
        self.nodes.append([start_node])

        u_list = []
        d_list = []
        for i in range(d['n']):
            u_list.append(self.u ** (i+1))
            d_list.append(self.d ** (i+1))

        for i in range(1, d['n']+1):
            self.nodes.append([])
            for j in range(i+1):
                a = i-j # new_S = St * u**(i-j) * d**(j)}
                b = j
                if a > b:
                    new_S = u_list[a-b-1] * d['St']
                elif a < b:
                    new_S = d_list[b-a-1] * d['St']
                else: # a==b
                    new_S = d['St']

                new_node = S_node(new_S)
                new_max_list = []
                new_a = i-j
                new_b = j
                while new_b >= 0:
                  # print("new_b = ", new_b)

                  if new_a > new_b:
                    to_added = u_list[new_a - new_b -1] * d['St']
                  elif new_a < new_b:
                    to_added = d_list[new_b - new_a -1] * d['St']
                  else:
                    to_added = d['St']

                  if to_added < d['Smax_t']:
                    to_added = d['Smax_t']

                  new_max_list.append(to_added)
                  new_b -= 1

                new_max_list = list(set(new_max_list))
                new_max_list.sort(reverse = True)
                new_node.set_max_list(new_max_list)
                self.nodes[i].append(new_node)

    def backward_induction(self):
        discnt = np.exp(- d['r'] * d['delta_t'])
        for i in range(d['n'],-1,-1):
          for j in range(len(self.nodes[i])):
            if i == d['n']:
              self.nodes[i][j].calculate_payoff(uparent_node=None, dparent_node=None, p = self.p, discount_factor = discnt, terminal = True)
            else:
              #print("-------------------------------------------")
              #print("i = %d, j = %d" % (i,j))
              self.nodes[i][j].calculate_payoff(uparent_node=self.nodes[i+1][j], dparent_node=self.nodes[i+1][j+1], \
                                                p = self.p, discount_factor = discnt, terminal = False)
        self.price_EP = self.nodes[0][0].value_EP[0]
        self.price_AP = self.nodes[0][0].value_AP[0]

    def print(self):
        for i in range(len(self.nodes)):
            for j in range(len(self.nodes[i])):
                print("i, j = %d, %d" % (i,j))
                self.nodes[i][j].print()

### Demo

In [ ]:
# Read files
d = {}

with open('HW4_params_3.txt','r') as file:
    line = file.readline().strip('\n')
    while line:
        key, value = line.split('=')[0], float(line.split('=')[1])
        d[key] = value
        line = file.readline().strip('\n')
d['n'] = int(d['n'])
d['sim_num'] = int(d['sim_num'])
d['rep_num'] = int(d['rep_num'])
d['delta_t'] = (d['T'] - d['t'])/d['n']
d

{'Smax_t': 70.0,
 'St': 50.0,
 'T': 0.25,
 'delta_t': 0.0025,
 'n': 100,
 'q': 0.0,
 'r': 0.1,
 'rep_num': 20,
 'sigma': 0.4,
 'sim_num': 10000,
 't': 0.0}

In [ ]:
# %%time
BTree = Binomial_tree()
BTree.build_tree()
BTree.backward_induction()
print("Binomial tree model: EP price = %4f, and AP price = %4f" % (BTree.price_EP, BTree.price_AP))
# %%time
sim_mean, sim_std = Monte_Carlo()
print("Monte Carlo:  EP price = %4f, and the C.I. = [%4f, %4f]"%(sim_mean, sim_mean - 2*sim_std, sim_mean + 2*sim_std))

Binomial tree model: EP price = 18.794611, and AP price = 20.000000
Monte Carlo:  EP price = 18.776202, and the C.I. = [18.588284, 18.964119]


In [ ]:
# %%time
BTree_1 = Binomial_tree()
BTree_1.build_tree_1()
BTree_1.backward_induction()
print("Binomial tree model: EP price = %4f, and AP price = %4f" % (BTree_1.price_EP, BTree_1.price_AP))

Binomial tree model: EP price = 7.236948, and AP price = 7.439574
